In [141]:
import csv
import numpy as np
import math

In [142]:
A = []
with open('data_logsumexp_A.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        row = [float(x) for x in row]
        # print(row)
        A.append(row)
A = np.array(A)
A.shape


(300, 100)

In [143]:
B = []
with open('data_logsumexp_B.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        B.append([float(x) for x in row])
B = np.array(B)
B.shape

(300, 1)

In [144]:
def f(x):
    return math.log(np.sum(np.exp((A @ x) + B)))

In [145]:
def f_diff1(x):
    results = np.empty_like(x)
    for i in range(A.shape[1]):
        a = A[:,i]
        # print('a:',a.shape)
        result = np.exp((A @ x) + B).reshape(A.shape[0],)
        result = (1/np.sum(np.exp((A @ x) + B))) * np.dot(result,a)
        results[i] = [result]
    return results
    # return (1/np.sum(np.exp((A @ x) + B)))*

In [146]:
def steepest_descent(x0,t=0.1,k=200):
    x_hist = []
    for i in range(k):
        if i == 0:
            x_pred = x0
        else:
            x_pred = x_hist[i-1]
        x_pred_delta = -1 * f_diff1(x_pred)
        x_hist.append(x_pred + (t * x_pred_delta))
    return x_hist

In [147]:
x = np.zeros((100,1))
x_hist = steepest_descent(x)
print(x_hist[0].shape)
print(len(x_hist))

(100, 1)
200
